In [18]:
import pandas as pd
import plotly.plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.graph_objs as go

In [19]:
#Read datasets. In this case, we use the excell file of tags to link pumps/level
pump = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4120/GBS_AR.RG4120_Instellingen_FT510_AM_Debietmeting.Q.csv", delimiter=";", decimal=",", dtype={'Value': float})
level = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4120/xGBS_AR.RG4110_Proces_LS411_Niveaumeting.Q.csv", delimiter=";", decimal=",", dtype={'Value': float})

In [20]:
#IMPORTANT: Parse dates
pump["TimeStamp"] = pd.to_datetime(pump['TimeStamp'], format='%d-%m-%Y %H:%M:%S')
level["TimeStamp"] = pd.to_datetime(level['TimeStamp'], format='%d-%m-%Y %H:%M:%S')

In [21]:
#Create an index and remove timestamp column and tag name
level.index = level["TimeStamp"]
pump.index = pump["TimeStamp"]

In [22]:
level.index.names = ['Date']
pump.index.names = ['Date']

In [23]:
pump = pump[["Value"]]
level = level[["Value"]]

In [25]:
#Cut the datasets:
level = level.loc['2017-08-07 00:00:05':]
pump = pump.loc[:'2018-08-06 23:42:05']

In [27]:
#now we need to resample so data is homogeneous
level = level.resample('30T').mean()
pump = pump.resample('30T').mean()

In [ ]:
flowdata = go.Scatter(
          x=pump.index,
          y=pump['Value'],
          name='flow')
leveldata = go.Scatter(
          x=level.index,
          y=level['Value'],
          name='Level',
          yaxis='y2')
data = [flowdata,leveldata]

In [ ]:
layout = go.Layout(
    title='Flow vs Level Aarle 4120',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1d',
                     step='day',
                     stepmode='backward'),
                dict(count=7,
                     label='1w',
                     step='day',
                     stepmode='backward'),
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),        
        rangeslider=dict(
            visible = True
        ),
        type='date'
    ),
    yaxis=dict(
        title='Flow (m3/h)'
    ),
    yaxis2=dict(
        title='Level (m above NAP)',
        overlaying='y',
        side='right'
    )
)
fig = dict(data=data, layout=layout)
offline.plot(fig, filename='4120.html')
py.iplot(fig)

In [ ]:
# COnclusion: original dataset only contains 0's and a few -1's :S